In [1]:
%%capture
from ovcDreams import ovcS_dreams as OVC, datim_ovc_dreams as DATIMOVC, regime_status as REGIMSTAT

In [6]:
from ovcDreams import positive_dreams

# HIVSTAT

In [2]:
OVC.test_results.value_counts()

0,       3489
0,,2,     361
2,        335
1,          2
Name: test_results, dtype: int64

In [3]:
OVC[OVC.test_results=="1,"]

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,number_of_condoms_sensibilize,number_condoms_reception_in_the_interval,number_test_date_in_the_interval,...,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ4,Gender
2816,100506,None,Ouest,Tabarre,0,0,no,1,0,0,...,no,no,no,no,no,no,no,yes,no,Female
4835,125012,None,Ouest,Port-au-Prince,0,0,no,1,0,0,...,no,secondary,no,no,complete,no,no,yes,yes,Female


In [5]:
REGIMSTAT[REGIMSTAT.id_patient.isin(OVC.id_patient)]

,patient_code,id_patient,start_date,regime,all_start_date,all_end_date


In [7]:
positive_dreams[positive_dreams.id_patient.isin(REGIMSTAT.id_patient)]

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,number_of_condoms_sensibilize,number_condoms_reception_in_the_interval,number_test_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ4
